### 简介
本版本更新于2021/2/13，相关参数说明如下：
* 人脸关键点识别方法：Baidu AI-人脸识别(待改进)
* 目标数据集格式：
 1. 特征：**整张面部图片** 标签：鼠标坐标（相对位置+归一化）
 2. 特征：左右脸宽度之比 & 鼻尖-眼部上侧高度差值 标签：鼠标坐标
* Python环境： python 3.6

In [6]:
#引入所有需要的模块，如有未安装的库建议pip install
# PyAutoGUI是一个纯Python的GUI自动化工具，其目的是可以用程序自动控制鼠标和键盘操作，多平台支持（Windows，OS X，Linux）。
#cv2即opencv2，用于实现电脑摄像头的调用等操作
#PIL 为Python 常用图像处理模块
import pyautogui as pag
import time
import requests
import os
import cv2
import urllib3,base64
import json
from PIL import Image
from urllib.parse import urlencode
import csv

###  一、收集并保存摄像头照片
通过openCV调用摄像头，在按下空格键时保存照片，命名为鼠标所在位置的元组形式。

运行此单元格后将弹出名为Capture的摄像头视频页面，敲击空格即可保存图片，通过ESC键终止运行。

In [3]:
#将此处改为初始数据集的保存路径
save_path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/'
#获取屏幕大小
screenWidth,screenHeight = pag.size()
#计算获得屏幕原点位置，定为屏幕中心点
origin = (screenWidth/2,screenHeight/2)

cap=cv2.VideoCapture(0) #调用摄像头，0为电脑自带摄像头，1为外部摄像头
while(1):
    ret,frame = cap.read()
    k=cv2.waitKey(1)
    if k==27: #Esc键退出
        break
    elif k==32:#空格键保存图片
        #获取当前鼠标绝对位置
        currMouseX,currMouseY = pag.position() 
        #将鼠标绝对位置转化为相对于原点的相对坐标，并进行归一化处理
        #对归一化处理有疑惑可参考：https://www.jianshu.com/p/95a8f035c86c
        currMouse = ((currMouseX-origin[0])/origin[0],(origin[1]-currMouseY)/origin[1])
        
        cv2.imwrite(save_path+str(currMouse)+'.jpg',frame)
        print(currMouse) #方便调试，若成功保存则会立刻在控制台输出
    cv2.imshow("capture", frame)
cap.release()
cv2.destroyAllWindows()

(0.14066666666666666, 0.414)


KeyboardInterrupt: 

### 二、调用人脸识别API获取关键点信息
调用百度AI-人脸识别以照片进行处理，并找出照片中的脸部关键点信息，以方便对图片信息进行下一步的处理。

BaiduAI 相关技术文档：https://ai.baidu.com/ai-doc/FACE/yk37c1u4t

注：此方法仅支持2QPS的查询率（即每秒2次的查询率），如需实际应用或用于实时演示则需更改此部分的代码或购买更大的处理能力。也可通过申请多个百度账号，建立账号池以增大qps。

In [7]:
# 首先通过百度AI提供的方法获取申请服务时的ID参数access_token，此参数至少一个月需更新一次，此处提供的为我的access_token
import requests 

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=QGsRMBXewEr9zcmapc3HeIVC&client_secret=PloEXIBs9Sf30tYVVtOppftGGgnqTwNh'
access_token = 'wait_to_get'
response = requests.get(host)
if response:
    access_token = response.json()['access_token']
    print(access_token)

24.2eb0dab646cd08942950a16d59db84d4.2592000.1615815823.282335-22192901


In [8]:
#BaiduAI 人脸识别方法调用，参数为文件路径，返回BaiduAI平台的反馈
def BaiduMethod(filepath):
    file = open(filepath,'rb')
    img = open(filepath,'rb')
    #参数images：图像base64编码 分别base64编码后的2张图片数据，需urlencode，半角逗号分隔，单次请求最大不超过20M
    img1 = base64.b64encode(img.read())
    request_url = "https://aip.baidubce.com/rest/2.0/face/v3/detect"+ "?access_token=" + access_token
    params = {'image':str(img1,'utf-8'),'image_type':'BASE64','face_field':'landmark'}
    headers = {'content-type': 'application/json'}
    return requests.post(request_url, data=params, headers=headers)

In [8]:
#此处改为所想读取的单个图片
filepath = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/(-0.5206666666666667, 0.013).jpg'
#考虑到百度AI平台的对并发量的限制，此处推荐分开处理，即获取response后不再重复对同一张照片进行处理
response = BaiduMethod(filepath)
if response:
    print (response.json())

{'error_code': 0, 'error_msg': 'SUCCESS', 'log_id': 7925354510199, 'timestamp': 1613221527, 'cached': 0, 'result': {'face_num': 1, 'face_list': [{'face_token': '7ceb0c9266de70bdb7e56d4f20f9f650', 'location': {'left': 277.74, 'top': 66.96, 'width': 197, 'height': 209, 'rotation': 2}, 'face_probability': 1, 'angle': {'yaw': -3.76, 'pitch': -6.47, 'roll': -0.05}, 'landmark': [{'x': 332.14, 'y': 109.03}, {'x': 426.11, 'y': 113.61}, {'x': 381.88, 'y': 149.83}, {'x': 376.32, 'y': 211.38}], 'landmark72': [{'x': 273.85, 'y': 137.52}, {'x': 275, 'y': 168.56}, {'x': 278.64, 'y': 199.33}, {'x': 286.85, 'y': 230}, {'x': 309.34, 'y': 259.37}, {'x': 340.03, 'y': 276.43}, {'x': 371.56, 'y': 281.08}, {'x': 401.48, 'y': 277.06}, {'x': 431.51, 'y': 260.88}, {'x': 452.94, 'y': 233.46}, {'x': 462.94, 'y': 204.26}, {'x': 468.68, 'y': 174.71}, {'x': 471.56, 'y': 145.52}, {'x': 310.94, 'y': 113.34}, {'x': 321.13, 'y': 105.67}, {'x': 332.19, 'y': 103.28}, {'x': 342.57, 'y': 105.49}, {'x': 352.52, 'y': 113.46}

API文档中用于对照参考的关键点识别示例照片：
https://ai.bdstatic.com/file/52BC00FFD4754A6298D977EDAD033DA0

In [2]:
#图像截取部分的代码，分离出来方便随时进行调整，此版本的设计可获取到近乎整张脸的照片。
def imgCrop(landmark,img): 
    box = (landmark[0]['x'],landmark[24]['y'],landmark[12]['x'],landmark[6]['y'])
    img2 = img.crop(box)
#     img2.show() #实时显示切割图片方便微调
    img3.save(save_path+file+'.jpg')
    
#数据集获取方式一，对应脸部照片-鼠标坐标的格式
def dataset1(path,save_path):
    dirs = os.listdir(path)
    for file in dirs:
        filepath = os.path.join(path,file)
        response = BaiduMethod(filepath)
        if response and response.json()['error_code']==0:#成功收到且未报错
            landmark = response.json()['result']['face_list'][0]['landmark72']
            img = Image.open(filepath)
            imgCrop(landmark,img)

#数据集获取方式一，对应面部参数-鼠标坐标的格式
def dataset2(path,save_path):
    dirs = os.listdir(path)
    rows = []
    for file in dirs:
        filepath = os.path.join(path,file)
        response = BaiduMethod(filepath)
        if response and response.json()['error_code']==0:#成功收到且未报错
            landmark = response.json()['result']['face_list'][0]['landmark72']
#           参数一：左半脸与右半脸宽度之比（相对观察者而言的左右）
            paramX = abs(landmark[0]['x']-landmark[57]['x'])/abs(landmark[12]['x']-landmark[57]['x'])
#           参数二：鼻尖与眼部上侧坐标纵坐标差值(建议进行进一步改进)
            eyeY = (landmark[15]['y'] + landmark[32]['y'])/2
            paramY = eyeY - landmark[57]['y']
#         (-0.5206666666666667, 0.013).jpg
            mouseX = file[1:file.index(',')]
            mouseY = file[file.index(',')+1:file.index(')')]
            #将四个参数保存到列表中
            rows.append([paramX,paramY,mouseX,mouseY])
    headers = ['paramX','paramY','mouseX','mouseY']
    with open('dataset.csv','w') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(rows)

In [9]:
path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet/' #待遍历目录路径
save_path = 'D:/UserData/Documents/Jupyter/EyeTrack/DataSet02/'#保存文件目录
dataset2(path,save_path) #